# Freedmen's Bureau Analysis 

## Task #0: Configure an LLM

### Import libraries that we'll use in this notebook

In [30]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_openai import OpenAI, ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, SystemMessage, AIMessage

import json


## Task #1: Parse Entitities from Simple Examples (Warmup)

### Samples to be able to parse correctly

In [31]:
inputs = [
    "John James agrees to pay $50/month to RJ Hampshire for work on the Farm",
    "Elizabeth James will pay $30 per month to Levi Rodgers for Gardening",
]

outputs = [
    {"payer": "John James", "payee": "RJ Hampshire", "amount": 50, "pay frequency": "monthly", "description": "farming"},
    {"payer": "Elizabeth James", "payee": "Levi Rodgers", "amount": 30, "pay frequency": "monthly", "description": "gardening"},
]

### Your code here

In [32]:
system_message = """
  From the following historical document text, please grab out the following items and return with the following format
  [{{
    "payer": "<Payer Name>",
    "payee": "<Payee Name>",
    "amount": <Amount Paid>, # This is the amount of money in USD
    "pay frequency": "<Frequency Paid>" # Only options are "monthly", "weekly", "yearly", "one-time", or "unknown"
    "description": "<Description of Work>" # This is the description of the work being done
  }},
  {{<ITEM 2>}}, {{Item 3>}}, ...]

  So for the following text:
  "Jane Doe will pay $50/month to John Smith for work on the Farm"

  the output text generated would be:
  [{{
    "payer": "Jane Doe",
    "payee": "John Smith",
    "amount": 50,
    "pay frequency": "monthly"
    "description": "work on the Farm"
  }}]

  IMPORTANT: Return only the list of Python dictionary objects and nothing more. It is possible there are no matches, in which case return an empty list.
  """

In [37]:
chat = ChatOpenAI()
# chat = ChatOllama(model="llama2")

messages = [SystemMessage(content=system_message), HumanMessage(content=inputs[0]), AIMessage(content=str(outputs[0])), HumanMessage(content=inputs[1])]

output = chat.invoke(messages)

print(output)


def process_response(response):
  response = json.loads(response)
  return response


content="[{'payer': 'Elizabeth James', 'payee': 'Levi Rodgers', 'amount': 30, 'pay frequency': 'monthly', 'description': 'Gardening'}]"


In [35]:
get_response(inputs[0], system_message=system_message)

KeyError: 'messages'